In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models

In [ ]:
# setting directories
ROOT_DIR = os.path.abspath('../input')
TRAIN_JPEG_DIR = os.path.join(ROOT_DIR, 'train-jpg')
TEST_JPEG_DIR = os.path.join(ROOT_DIR, 'test-jpg-v2')
TRAIN_TIF_DIR = os.path.join(ROOT_DIR, 'train-tif-v2')
TEST_TIF_DIR = os.path.join(ROOT_DIR, 'test-tif-v3')

In [ ]:
def read_image(path):
    im = cv2.imread(path)
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

In [ ]:
def normalize(im):
    """Normalizes images with Imagenet stats."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0]) / imagenet_stats[1]

In [ ]:
training_images = os.listdir(TRAIN_JPEG_DIR)
print(training_images[0])

In [ ]:
sample_path = os.path.join(TRAIN_JPEG_DIR, 'train_9203.jpg')
im = read_image(sample_path)
plt.imshow(im)

In [ ]:
# typical size: 256x256 so should be able to train directly without resizing
dims = [read_image(os.path.join(TRAIN_JPEG_DIR, p)).shape for p in training_images[:50]]
dims[:10]

### Data splitting

In [ ]:
train_file_names = list(os.listdir(TRAIN_JPEG_DIR))
train_labels_df = pd.read_csv(os.path.join(ROOT_DIR, 'train_v2.csv'))
test_file_names = list(os.listdir(TEST_JPEG_DIR))

In [ ]:
train_IDs = [f.split('.')[0] for f in train_file_names]
test_IDs = [f.split('.')[0] for f in test_file_names]

In [ ]:
inner_train_IDs, val_IDs = train_test_split(train_IDs, test_size=0.2, random_state=42)

In [ ]:
partition = {'train': train_IDs, 'inner_train': inner_train_IDs, 
             'validation': val_IDs, 'test': test_IDs}

In [ ]:
train_labels = []
for tag in train_labels_df.tags:
    train_labels.append(tag.split())

In [ ]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_labels)

In [ ]:
labels = {}
for i, row in enumerate(train_labels_df.itertuples()):
    labels[row.image_name] = train_labels[i]

### Data Augmentation

In [ ]:
def crop(im, r, c, target_r, target_c):
    return im[r:r+target_r, c:c+target_c]

# random crop to the original size
def random_crop(x, r_pix=8):
    """Returns a random crop"""
    r, c, *_ = x.shape
    r, c, *_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    return crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)

def center_crop(x, r_pix=8):
    r, c, *_ = x.shape
    c_pix = round(r_pix*c/r)
    return crop(x, r_pix, c_pix, r-2*r_pix, c-2*c_pix)


def rotate_cv(im, deg, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r, c, *_ = im.shape
    M = cv2.getRotationMatrix2D((c/2, r/2), deg, 1)
    return cv2.warpAffine(im, M, (c,r), borderMode=mode, 
                          flags=cv2.WARP_FILL_OUTLIERS+interpolation)

### Dataset

In [ ]:
class PlanetDataset(Dataset):
    def __init__(self, folder_path, list_IDs, labels, transforms=False):
        self.list_IDs = list_IDs
        self.labels = labels
        self.folder_path = folder_path
        self.transforms = transforms
    
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, idx):
        name = self.list_IDs[idx]
        file_path = os.path.join(self.folder_path, name + '.jpg')
        x = cv2.imread(file_path).astype(np.float32)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) / 255
        if self.transforms:
            rdeg = (np.random.random() - 0.50) * 20
            x = rotate_cv(x, rdeg)
            x = random_crop(x)
            if np.random.random() > 0.5:
                x = np.fliplr(x).copy()
            else:
                x = center_crop(x)
        x = normalize(x)
        if self.labels is not None:
            y = torch.from_numpy(self.labels[name])
            return np.rollaxis(x, 2), y
        return np.rollaxis(x, 2)

In [ ]:
inner_train_ds = PlanetDataset(TRAIN_JPEG_DIR, partition['inner_train'], labels)
val_ds = PlanetDataset(TRAIN_JPEG_DIR, partition['validation'], labels)

In [ ]:
batch_size = 64
inner_train_dl = DataLoader(inner_train_ds, batch_size=batch_size, shuffle=True) 
val_dl = DataLoader(val_ds, batch_size=batch_size) 

### Models

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        resnet = models.resnet34(pretrained=True)
        # freezing parameters
        for param in resnet.parameters():
            param.requires_grad = False
        # convolutional layers of resnet34
        layers = list(resnet.children())[:8]
        self.top_model = nn.Sequential(*layers).cuda()
        self.fc = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = F.relu(self.top_model(x))
        x = nn.AdaptiveAvgPool2d((1, 1))(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
net = Net(num_classes=17).cuda()

In [ ]:
x, y = iter(inner_train_dl).next()
x = x.cuda().float()
y = y.cuda().float()

In [ ]:
output = net(x)

In [ ]:
output_sigmoid = F.sigmoid(output)

In [ ]:
output_sigmoid.cpu().detach().numpy()

In [ ]:
fbeta_score(y.cpu(), output_sigmoid.cpu() > 0.5, beta=2, average='macro')

In [ ]:
y

In [ ]:
F.binary_cross_entropy_with_logits(y, output)

### Train model

In [ ]:
def val_metrics(model, val_dl):
    model.eval()
    total = 0
    sum_loss = 0
    sum_f2 = 0
    
    for x, y in val_dl:
        batch = y.shape[0]
        x = x.cuda().float()
        y = y.cuda().float()
        out = model(x)
        loss = F.binary_cross_entropy_with_logits(out, y)
        sum_loss += batch*(loss.item())
        pred = F.sigmoid(out) > 0.5
        sum_f2 += batch*(fbeta_score(y.cpu(), pred.cpu() > 0.5, beta=2, average='macro'))
        total += batch
    return sum_loss / total, sum_f2 / total

In [ ]:
val_metrics(net, val_dl)

In [ ]:
def train(model, train_dl, val_dl, epochs=10, learning_rate=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        model.train()
        total = 0
        sum_loss = 0
        for x, y in train_dl:
            batch_size = y.shape[0]
            x = x.cuda().float()
            y = y.cuda().float()
            out = model(x)
            loss = F.binary_cross_entropy_with_logits(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total += batch_size
            sum_loss += batch_size*(loss.item())
        train_loss = sum_loss / total
        val_loss, val_f2 = val_metrics(model, val_dl)
        print("Epoch: %d, Train loss: %.3f, val loss: %.3f, val f2: %.3f" % (epoch+1, train_loss, val_loss, val_f2))

In [ ]:
train_labels

In [ ]:
net = Net(num_classes=17).cuda()

In [ ]:
batch_size = 64
inner_train_dl = DataLoader(inner_train_ds, batch_size=batch_size, shuffle=True) 
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [ ]:
train(net, inner_train_dl, val_dl)